In [79]:
import os
import pickle

import numpy as np
import pandas as pd
import torch
from matplotlib import pyplot as plt
from numpy import interp
from sklearn import svm, metrics
from sklearn.decomposition import KernelPCA, PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import auc, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import xgboost as xgb

In [80]:
# 1 加载数据
train_X_df = pd.read_excel(r"D:\GitHub\modeling\D\ans3\Molecular_Descriptor.xlsx", engine='openpyxl',
                           sheet_name='training')
train_Y_df = pd.read_excel(r"D:\GitHub\modeling\D\ans3\ADMET.xlsx", engine='openpyxl', sheet_name='training')

In [81]:
var_not_null = []

In [82]:
# # 只用第一问的20个特征
# var = ['MDEC-23', 'minsssN', 'LipoaffinityIndex', 'maxHsOH', 'maxssO', 'C1SP2',
#                    'minHsOH', 'BCUTc-1l', 'minsOH', 'minHBint5', 'MLFER_A', 'nHBAcc', 'VC-5',
#                    'MDEO-12', 'ndssC', 'TopoPSA', 'ATSc3', 'SHBint10', 'MDEC-33', 'XLogP']
# var_top20 = pd.Index(var)
# # var_top20 = pd.Index(var_top20)
# # X_train_df = train_X_df[var_top20]
# X_train_df = train_X_df
# 找出全零的列，将其剔除
# idx_df = pd.read_excel(r"D:\GitHub\modeling\D\ans3\idx_not_null.xlsx", engine='openpyxl', sheet_name='Sheet1')
# idx = idx_df.values.tolist()
# idx_one = [b for a in idx for b in a]
# X_train_df = train_X_df[idx_one]

X = train_X_df.iloc[:, 1:].values  # 去掉第一列
Y = train_Y_df.iloc[:, 1].values  # 第一个label


# idx_series = pd.Series(idx_df[:,:].values)
# df_not_null = X_train_df.loc[:, idx_series]
# print(df_not_null.shape)

In [83]:
# print(X_train_df.shape)
# idx = (X_train_df != 0).any(axis=0)
# # 编码指定为UTF-8否则读取的时候会报错
# idx.to_csv("idx_not_null.csv", header='true', encoding='utf-8')
# df = X_train_df.loc[:, (X_train_df != 0).any(axis=0)]
# # X_train_df = X_train_df.loc[~(X_train_df==0).all(axis=0), :]
# print(df.shape)

In [84]:
# 2 降维
# scikit_kpca = KernelPCA()
#
# # 使用KPCA降低数据维度，直接获得投影后的坐标
# X = scikit_kpca.fit_transform(X)


In [85]:
# 3 分割训练数据和测试数据
# random_state 设置随机数为33
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=33)

# 3 在训练之前，我们需要对数据进行规范化，这样让数据同在同一个量级上，避免因为维度问题造成数据误差：
# 采用 Z-Score 规范化数据，保证每个特征维度的数据均值为 0，方差为 1
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

nag_sum = np.sum(y_train==0)
pos_sum = np.sum(y_train==1)

In [86]:
# 4 创建 SVM 分类器
# model_svm = svm.SVC(kernel='poly',degree=8, gamma='auto')
model_svm = svm.SVC(decision_function_shape="ovr",probability=True)
# 用训练集做训练
model_svm.fit(x_train, y_train)
# 用测试集做预测
p_svm = model_svm.predict(x_test)
print('svm准确率: ', metrics.accuracy_score(p_svm, y_test))
test_predict_label = model_svm.predict_proba(x_test)
print(test_predict_label.shape)
print(model_svm.classes_)

svm准确率:  0.9139240506329114
(395, 2)
[0 1]


In [87]:
def modelfit(alg, x_train, y_train, x_test, y_test, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(x_train, label=y_train)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])

    #Fit the algorithm on the data
    alg.fit(x_train, y_train,eval_metric='auc')

    #Predict training set:
    dtrain_predictions = alg.predict(x_test)
    dtrain_predprob = alg.predict_proba(x_test)[:,1]

    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(y_test, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(y_test, dtrain_predprob))

    # feat_imp = pd.Series(alg.get_booster().get_fscore()).sort_values(ascending=False)[:20]
    # feat_imp.plot(kind='bar', title='Feature Importances')
    # plt.ylabel('Feature Importance Score')


In [88]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=nag_sum/pos_sum,
 seed=27)
modelfit(xgb1, x_train, y_train, x_test, y_test)


C:\Users\nuo_nuaa\anaconda3\envs\diveintoDL\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



Model Report
Accuracy : 0.9114
AUC Score (Train): 0.974734


In [89]:
#Choose all predictors except target & IDcols
xgb_test = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=7,
 min_child_weight=1,
 gamma=0,
 subsample=0.7,
 colsample_bytree=0.7,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=nag_sum/pos_sum,
 seed=27)

# param_test1 = {
#  'max_depth':range(3,10,2),
#  'min_child_weight':range(1,6,2)
# }
# param_test3 = {
#  'gamma':[i/10.0 for i in range(0,5)]
# }
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch1 = GridSearchCV(estimator = xgb_test, param_grid = param_test4 , scoring='roc_auc',n_jobs=4, cv=5)
# gsearch1.fit(x_train,y_train)
# gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_


In [90]:
xgb2 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=7,
 min_child_weight=1,
 gamma=0,
 subsample=0.7,
 colsample_bytree=0.7,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=nag_sum/pos_sum,
 seed=27,
)
modelfit(xgb2, x_train, y_train, x_test, y_test)
y_pred_xgb = xgb2.predict_proba(x_test)

C:\Users\nuo_nuaa\anaconda3\envs\diveintoDL\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



Model Report
Accuracy : 0.9165
AUC Score (Train): 0.977014


In [91]:
# 5 xgboost
# model_xgboost = XGBClassifier(learning_rate=0.01,
#                       n_estimators=10,           # 树的个数-10棵树建立xgboost
#                       max_depth=4,               # 树的深度
#                       min_child_weight = 1,      # 叶子节点最小权重
#                       gamma=0.,                  # 惩罚项中叶子结点个数前的参数
#                       subsample=1,               # 所有样本建立决策树
#                       colsample_btree=1,         # 所有特征建立决策树
#                       scale_pos_weight=1,        # 解决样本个数不平衡的问题
#                       random_state=27,           # 随机数
#                       slient = 0
#                       )

# nag_sum = np.sum(y_train==0)
# pos_sum = np.sum(y_train==1)
model_xgboost = XGBClassifier(booster='gbtree', scale_pos_weight=nag_sum/pos_sum)
model_xgboost.fit(x_train, y_train)
# 用测试集做预测
p_xgboost = model_xgboost.predict(x_test)
print('xgboost准确率: ', metrics.accuracy_score(p_xgboost, y_test))

[21:35:26] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\nuo_nuaa\anaconda3\envs\diveintoDL\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


xgboost准确率:  0.9113924050632911


In [92]:
# # 测试结果
# test_X_df = pd.read_excel(r"D:\GitHub\modeling\D\ans3\Molecular_Descriptor.xlsx", engine='openpyxl',
#                            sheet_name='test')
# # test_X_df = test_X_df[idx_one]
# test_X = test_X_df.iloc[:, 1:].values  # 去掉第一列
# test_X = ss.fit_transform(test_X)
# pred = model_xgboost.predict(test_X)
# # list转dataframe
# col_name = 'Caco-2'
# df = pd.DataFrame(pred, columns=[col_name])
# # 保存到本地excel
# df.to_excel(col_name + ".xlsx", index=False)

# save model to file
# pickle.dump(model_xgboost, open(col_name + ".pickle", "wb"))

In [93]:
# 6 随机森林
# model_rf = RandomForestClassifier(n_estimators = 1000, max_depth=None, random_state=0)
model_rf = RandomForestClassifier()
model_rf.fit(x_train, y_train)
# 用测试集做预测
p_rf = model_rf.predict(x_test)


print('随机森林准确率: ', metrics.accuracy_score(p_rf, y_test))
rf_pre = model_rf.predict_proba(x_test)

随机森林准确率:  0.9012658227848102


In [94]:
model_1 = pickle.load(open(r'D:\GitHub\modeling\D\ans3\Caco-2.pickle', "rb"))
pred_1 = model_1.predict(x_test)
print('确率: ', metrics.accuracy_score(pred_1, y_test))

确率:  0.9113924050632911


In [95]:
# model_1 = pickle.load(open(r'D:\GitHub\modeling\D\ans3\Caco-2.pickle', "rb"))
# pred_2 = model_1.predict(x_train)
# print('确率: ', metrics.accuracy_score(pred_2, y_train))
# auc_saved_path = 'p_xgboost.pkl'
# fpr, tpr, threshold = roc_curve(p_xgboost, y_test)  ###计算真正率和假正率
# roc_auc = auc(fpr, tpr)  ###计算auc的值
# print(fpr.shape, tpr.shape, threshold.shape)
# print('fpr= ', fpr)
# print('tpr= ', tpr)
# print('threshold= ', threshold)
# data_roc = dict()
# data_roc['micro-auc'] = roc_auc
# data_roc['fpr-micro'] = fpr
# data_roc['tpr-micro'] = tpr
#
# output = open(auc_saved_path, 'wb')
# pickle.dump(data_roc, output)
# output.close()

In [96]:
# num_class =2
# fpr_dict = dict()
# tpr_dict = dict()
# roc_auc_dict = dict()
# auc_saved_path = 'p_xgboost.pkl'
# # micro
# fpr_dict["micro"], tpr_dict["micro"], _ = roc_curve(p_xgboost, y_test)
# roc_auc_dict["micro"] = auc(fpr_dict["micro"], tpr_dict["micro"])
#
# # macro
# # First aggregate all false positive rates
# all_fpr = np.unique(np.concatenate([fpr_dict[i] for i in range(num_class)]))
# # Then interpolate all ROC curves at this points
# mean_tpr = np.zeros_like(all_fpr)
# for i in range(num_class):
#     mean_tpr += interp(all_fpr, fpr_dict[i], tpr_dict[i])
# # Finally average it and compute AUC
# mean_tpr /= num_class
# fpr_dict["macro"] = all_fpr
# tpr_dict["macro"] = mean_tpr
# roc_auc_dict["macro"] = auc(fpr_dict["macro"], tpr_dict["macro"])
# print('micro-auc: ', roc_auc_dict["micro"])
# print('macro-auc: ', roc_auc_dict["macro"])
# print('0-auc: ', roc_auc_dict[0])
# print('1-auc: ', roc_auc_dict[1])
#
# data_roc = dict()
# data_roc['0-auc'] = roc_auc_dict[0]
# data_roc['1-auc'] = roc_auc_dict[1]
# data_roc['micro-auc'] = roc_auc_dict["micro"]
# data_roc["macro-auc"] = roc_auc_dict["macro"]
# data_roc['fpr-micro'] = fpr_dict["micro"]
# data_roc['tpr-micro'] = tpr_dict["micro"]
# data_roc['fpr-macro'] = fpr_dict["macro"]
# data_roc['tpr-macro'] = tpr_dict["macro"]
#
# output = open(auc_saved_path, 'wb')
# pickle.dump(data_roc, output)
# output.close()

In [99]:
# score_list = test_predict_label  # svm
# score_list = y_pred_xgb  # xgb
score_list = rf_pre  # xgb

label_list = y_test
num_class = 2
auc_saved_path = 'p_rf.pkl'
# 统计roc相关
score_array = np.array(score_list)
# 将label转换成onehot形式
label_tensor = torch.tensor(label_list)
label_tensor = label_tensor.reshape((label_tensor.shape[0], 1))
label_onehot = torch.zeros(label_tensor.shape[0], num_class)
label_onehot.scatter_(dim=1, index=label_tensor, value=1)
label_onehot = np.array(label_onehot)
# 调用sklearn库，计算每个类别对应的fpr和tpr
num_class =2
fpr_dict = dict()
tpr_dict = dict()
roc_auc_dict = dict()
for i in range(num_class):
    fpr_dict[i], tpr_dict[i], _ = roc_curve(label_onehot[:, i], score_array[:, i])
    roc_auc_dict[i] = auc(fpr_dict[i], tpr_dict[i])
# micro
fpr_dict["micro"], tpr_dict["micro"], _ = roc_curve(label_onehot.ravel(), score_array.ravel())
roc_auc_dict["micro"] = auc(fpr_dict["micro"], tpr_dict["micro"])

# macro
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr_dict[i] for i in range(num_class)]))
# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(num_class):
    mean_tpr += interp(all_fpr, fpr_dict[i], tpr_dict[i])
# Finally average it and compute AUC
mean_tpr /= num_class
fpr_dict["macro"] = all_fpr
tpr_dict["macro"] = mean_tpr
roc_auc_dict["macro"] = auc(fpr_dict["macro"], tpr_dict["macro"])
print('micro-auc: ', roc_auc_dict["micro"])
print('macro-auc: ', roc_auc_dict["macro"])
print('0-auc: ', roc_auc_dict[0])
print('1-auc: ', roc_auc_dict[1])

data_roc = dict()
data_roc['0-auc'] = roc_auc_dict[0]
data_roc['1-auc'] = roc_auc_dict[1]
data_roc['micro-auc'] = roc_auc_dict["micro"]
data_roc["macro-auc"] = roc_auc_dict["macro"]
data_roc['fpr-micro'] = fpr_dict["micro"]
data_roc['tpr-micro'] = tpr_dict["micro"]
data_roc['fpr-macro'] = fpr_dict["macro"]
data_roc['tpr-macro'] = tpr_dict["macro"]

output = open(auc_saved_path, 'wb')
pickle.dump(data_roc, output)
output.close()

micro-auc:  0.9689472840890881
macro-auc:  0.967691233874891
0-auc:  0.9671843149876622
1-auc:  0.9671843149876623
